In [ ]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import os
import datetime as dt
import plotly.express as px

from config import BASE_DIR
print(os.getcwd())
os.chdir(BASE_DIR)
print(BASE_DIR)

import scripts.functions as fn

from scripts.indicators import Fibonacci
from scripts.indicators import resample, zigzag, fibonacci_retroceso, fibonacci_extension

from binance.client import Client

C:\Users\lbisa\Mi unidad (leonardo.bisaro@gmail.com)\Cripto\python\trd_app
C:\Users\lbisa\Mi unidad (leonardo.bisaro@gmail.com)\Cripto\python\trd_app


In [16]:
#Funcion para descarga de velas
def get_klines(symbol,interval,start,end):
    print('Descargando velas',symbol, interval, start, end )
    df = client.get_historical_klines(symbol=symbol, interval=interval, start_str=start, end_str=end)
    df = pd.DataFrame(df)
    df = df.iloc[:, :6]
    df.columns = ["datetime", "open", "high", "low", "close", "volume"]
    df['open'] = df['open'].astype('float')
    df['high'] = df['high'].astype('float')
    df['low'] = df['low'].astype('float')
    df['close'] = df['close'].astype('float')
    df['volume'] = df['volume'].astype('float')
    df['datetime'] = pd.to_datetime(df['datetime'], unit='ms') - dt.timedelta(hours=3)
    #df=df.set_index('datetime')
    return df

In [17]:
environment = 'klines'
environment = 'binance'

if environment == 'klines':

	klines_file  = './backtest/klines/0m01/Completo_BTCUSDT_0m01_2021-01-01_2023-07-31.DataFrame'
	with open(klines_file, 'rb') as file:
		df = pickle.load(file)
		#df.set_index('datetime',inplace=True)

	df = resample(df,(60))
else:
	#Conexion con Binance sin API-KEY
	client = Client()

	start = '2025-01-01'
	end = '2025-05-14'
	symbol = 'BTCUSDT'
	interval = '1h'
	real_start = (dt.datetime.strptime(start, "%Y-%m-%d") ).strftime("%Y-%m-%d")
	real_end = (dt.datetime.strptime(end, "%Y-%m-%d") ).strftime("%Y-%m-%d")
	utc_start = real_start+' 00:00:00'
	utc_end = real_end+' 23:59:59'
	df = get_klines(symbol=symbol,interval=interval, start=utc_start,end=utc_end)

#df.tail()

Descargando velas BTCUSDT 1h 2025-01-01 00:00:00 2025-05-14 23:59:59


In [18]:


# Supongamos que tienes un DataFrame llamado df con datos históricos de 1 hora
# Asegúrate de que la columna 'datetime' esté en formato datetime
df['datetime'] = pd.to_datetime(df['datetime'])

# Extraer la hora del día de la columna 'datetime'
df['hour'] = df['datetime'].dt.hour

# Agrupar por hora y calcular el volumen promedio
hourly_volume = df.groupby('hour')['volume'].mean().reset_index()

# Crear un gráfico interactivo con Plotly
fig = px.bar(
    hourly_volume,
    x='hour',
    y='volume',
    title='Volumen promedio por hora del día',
    labels={'hour': 'Hora del día', 'volume': 'Volumen promedio'},
    text='volume',
)

# Ajustar el formato del gráfico
fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig.update_layout(
    xaxis=dict(tickmode='linear', tick0=0, dtick=1),  # Mostrar cada hora en el eje X
    yaxis=dict(title='Volumen promedio'),
    bargap=0.2,
    template='plotly_white'
)

# Mostrar el gráfico
fig.show()
